# Análisis Estadístico Multivariado
# 29 de Noviembre de 2025
# Proyecto final
# Acoyani Garrido Sandoval

## 1. Descripción

Consiste en determinar la relación entre una variable dependiente y una o más variables independientes a través de regresión logística múltiple. Se usará para ello un conjunto de datos de patrones de órdenes de comida rápida en diferentes ciudades de India, desarrollado por Prince Rajak y disponible en [Kaggle](https://www.kaggle.com/datasets/prince7489/fast-food-ordering-pattern-dataset).



In [1]:
import pandas
import numpy

## 2. Análisis descriptivo

Comenzamos por desarrollar un análisis descriptivo de las variables del conjunto de datos.

Cada registro de nuestro conjunto de datos es un pedido individual de comida rápida, cuyas variables son:
- **order_id:** Número de pedido
- **order_time:** Fecha y hora del pedido en tiempo de India
- **city:** Ciudad donde fue hecho el pedido
- **cuisine_type:** Estilo de cocina
- **order_value:** Precio del pedido en rupias
- **delivery_time_minutes:** Tiempo del pedido en minutos
- **items_count:** Cuántos elementos tiene el pedido
- **payment_method:** Método de pago:
   - **UPI:** *Unified Payments Interface,* un sistema de pagos electrónicos operado por el banco nacional de India
   - **Wallet:** pagos hechos mediante sistemas orientados a dispositivos móviles, tales como Apple y Google Pay.
   - **Credit Card:** tarjeta de crédito
   - **Debit Card:** tarjeta de débito
   - **Cash:** efectivo

El análisis que haremos incluirá:

1. **Para las variables de razón:** Medidas de tendencia central y de dispersión: moda, media, mediana, rango, desviación estándar, rango intercuartílico, y diagrama de caja.
   - Minutos de entrega
   - Cantidad de elementos
   - Precio
1. **Para la variable de intervalo:** Distribución (histograma), moda y mediana
   - Hora del pedido
1. **Para las variables nominales:** Conteo en gráfica, y porcentaje
   - Ciudad
   - Estilo de cocina
   - Método de pago
